In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import os
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import deque
import copy
import pandas as pd

In [5]:
model = load_model("/kaggle/input/detection-model/tensorflow2/default/1/best_model_high_res_512_grayscale (1).h5")
last_conv_layer = "last_conv_layer"
segmentation_model = load_model("/kaggle/input/segmentation_model/tensorflow2/default/1/trained_model.hdf5", compile=False)

In [6]:
def get_arr_img_segmentation(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=-1) 
    img = np.expand_dims(img, axis=0)
    return img
    
def get_arr_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (512, 512))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=-1) 
    img = np.expand_dims(img, axis=0)
    return img

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = tf.image.resize(
        heatmap[..., tf.newaxis],  # Add channel dim (now shape: [H, W, 1])
        (512, 512),
        method=tf.image.ResizeMethod.BILINEAR  # Smooth interpolation
    )
    return heatmap.numpy()

def make_gradcam_plus_plus_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape1:
        with tf.GradientTape() as tape2:
            with tf.GradientTape() as tape3:
                conv_output, predictions = grad_model(img_array)
                if pred_index is None:
                    pred_index = tf.argmax(predictions[0])
                target_class_score = predictions[:, pred_index]
            grads = tape3.gradient(target_class_score, conv_output)
        grads2 = tape2.gradient(grads, conv_output)
    grads3 = tape1.gradient(grads2, conv_output)

    # Compute weights
    first_term = tf.square(grads)
    second_term = grads2
    third_term = conv_output

    # Avoid division by zero
    denominator = 2 * grads2 + grads3 * third_term
    denominator = tf.where(denominator != 0.0, denominator, tf.ones_like(denominator))

    alphas = first_term / denominator
    alpha_normalization_constant = tf.reduce_sum(alphas, axis=(1, 2), keepdims=True)
    alphas /= alpha_normalization_constant

    # ReLU of gradients
    relu_grads = tf.nn.relu(grads)

    # Weight the channels by alpha * relu gradient
    weights = tf.reduce_sum(alphas * relu_grads, axis=(1, 2))

    # Weighted combination of forward activations
    cam = tf.reduce_sum(weights[:, tf.newaxis, tf.newaxis, :] * conv_output, axis=-1)
    heatmap = tf.squeeze(cam)

    # Normalize
    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.reduce_max(heatmap)
    if max_val != 0:
        heatmap /= max_val
    heatmap = tf.image.resize(
        heatmap[..., tf.newaxis],  # shape [H, W, 1]
        (512, 512),  # adjust size as needed
        method=tf.image.ResizeMethod.BILINEAR
    )

    return heatmap.numpy()

def compute_occlusion_sensitivity(
    img_array, model, patch_size=64, stride=16, baseline_value=0.5):
    
    input_img = img_array.copy()
    H, W = input_img.shape[1], input_img.shape[2]
    heatmap = np.zeros((H, W))

    # Get the original prediction
    original_pred = model.predict(img_array, verbose=0)[0][0]

    for y in range(0, H, stride):
        for x in range(0, W, stride):
            # Create a copy and occlude the patch
            occluded_img = input_img.copy()
            y1, y2 = y, min(y + patch_size, H)
            x1, x2 = x, min(x + patch_size, W)
            occluded_img[0, y1:y2, x1:x2, :] = baseline_value

            # Get new prediction
            pred = model.predict(occluded_img, verbose=0)[0][0]

            # The drop in prediction score is used as importance
            score_drop = original_pred - pred
            heatmap[y1:y2, x1:x2] += score_drop

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    # Resize to match original image
    heatmap = tf.image.resize(heatmap[..., np.newaxis], (512, 512)).numpy().squeeze()
    return heatmap

def scorecam_heatmap(img_array, model, last_conv_layer_name, class_index=None):
    
    conv_layer_model = keras.Model(model.inputs, model.get_layer(last_conv_layer_name).output)
    conv_output = conv_layer_model(img_array)  # Shape: (1, H, W, N)
    conv_output = conv_output[0]  # Shape: (H, W, N)

    pred = model(img_array)
    if class_index is None:
        class_index = tf.argmax(pred[0])

    heatmap = np.zeros(shape=conv_output.shape[:2], dtype=np.float32)  
    for i in range(conv_output.shape[-1]):
        activation = conv_output[..., i]

        # Normalize activation to [0, 1]
        activation -= tf.reduce_min(activation)
        activation /= tf.reduce_max(activation) + 1e-10
        upsampled = tf.image.resize(activation[..., tf.newaxis], (img_array.shape[1], img_array.shape[2]))
        upsampled_img = upsampled.numpy()

        # Multiply original image by upsampled activation map (masking)
        masked_input = img_array * upsampled_img

        # Get score for the target class
        score = model(masked_input)[0, class_index].numpy()
        heatmap += score * activation.numpy()

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-10

    # Resize to input image size (if needed)
    heatmap = tf.image.resize(heatmap[..., np.newaxis], (img_array.shape[1], img_array.shape[2]))
    return heatmap.numpy().squeeze()

def make_smoothgrad_heatmap(img_array, model, class_index=None, n_samples=50, noise_level=0.2):
    if class_index is None:
        pred = model(img_array)
        class_index = tf.argmax(pred[0])

    grads_list = []

    for _ in range(n_samples):
        noise = tf.random.normal(shape=img_array.shape, stddev=noise_level)
        noisy_input = img_array + noise
        with tf.GradientTape() as tape:
            tape.watch(noisy_input)
            preds = model(noisy_input)
            loss = preds[:, class_index]
        grads = tape.gradient(loss, noisy_input)  # Shape: (1, H, W, 1)
        grads_list.append(grads[0])  # Remove batch dim

    # Average the gradients
    avg_grads = tf.reduce_mean(tf.stack(grads_list), axis=0)  # Shape: (H, W, 1)
    avg_grads = tf.abs(avg_grads)  # Optional: Take absolute value for visualization

    # Normalize to [0, 1]
    heatmap = avg_grads - tf.reduce_min(avg_grads)
    heatmap /= tf.reduce_max(heatmap) + 1e-10

    # Resize to match original image shape
    heatmap = tf.image.resize(heatmap, (img_array.shape[1], img_array.shape[2]))

    return heatmap.numpy().squeeze()

def make_integrated_gradients_heatmap(
    img_array,
    model,
    class_index=None,
    baseline=None,
    steps=50
):
        # Convert input to float32 if it's not already
    img_array = tf.cast(img_array, tf.float32)
    
    # If baseline is not provided, use a black image
    if baseline is None:
        baseline = tf.zeros_like(img_array)
    else:
        baseline = tf.cast(baseline, tf.float32)
    
    # Interpolate inputs between baseline and actual image
    alphas = tf.linspace(0.0, 1.0, steps + 1)
    interpolated = [baseline + alpha * (img_array - baseline) for alpha in alphas]
    interpolated = tf.concat(interpolated, axis=0)  # Shape: (steps+1, H, W, 1)
    
    # Compute gradients
    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        preds = model(interpolated)
        class_logits = preds[:, class_index]
    
    grads = tape.gradient(class_logits, interpolated)
    
    # Approximate the integral using the trapezoidal rule
    grads = (grads[:-1] + grads[1:]) / 2.0
    avg_grads = tf.reduce_mean(grads, axis=0)
    
    # Calculate integrated gradients
    integrated_grads = (img_array - baseline) * avg_grads
    
    # Normalize to [0, 1]
    heatmap = avg_grads - tf.reduce_min(avg_grads)
    heatmap /= tf.reduce_max(heatmap) + 1e-10

    # Resize to match original image shape
    heatmap = tf.image.resize(heatmap, (img_array.shape[1], img_array.shape[2]))

    return heatmap.numpy().squeeze()

In [7]:
# 300 random images
images_arr = []
with open("/kaggle/input/random-paths/random_paths.txt", "r") as file:
    for line in file:
        path = line.strip()
        arr = get_arr_img(path)
        images_arr.append(arr)

print(f"{len(images_arr)} images" )

300 images


In [8]:
headers = ["Robustness", "Faithfulness", "Complexity", "Randomization"]
methods = [
    "Grad-CAM", 
    "Grad-CAM++", 
    "Score-CAM", 
    "Integrated Gradients", 
    "SmoothGrad", 
    "Occlusion Sensitivity",
    "GradCAM + Lung segment"
]
df = pd.DataFrame(index=methods, columns=headers)
df.fillna("-", inplace=True)  
display(df)

,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,-,-,-,-
Grad-CAM++,-,-,-,-
Score-CAM,-,-,-,-
Integrated Gradients,-,-,-,-
SmoothGrad,-,-,-,-
Occlusion Sensitivity,-,-,-,-
GradCAM + Lung segment,-,-,-,-


<center><h1>Grad-CAM</h1></center>

In [10]:
#________________________________________________________________________________________________________________________GRAD-CAM
#____________________________________________________________________________________________ROBUSTNESS
ab = []
ab = images_arr
def explanation_robustness(original_explanation, image):
    epsilon=0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = make_gradcam_heatmap(perturbed_img, model, last_conv_layer)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)
scores = []
    
for arr in ab:
    heatmap = make_gradcam_heatmap(arr, model, last_conv_layer)
    score = explanation_robustness(heatmap, arr)
    scores.append(score)    
    
        
df.at["Grad-CAM", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image):        
    k=0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    heatmap = make_gradcam_heatmap(image, model, last_conv_layer)
    threshold = np.percentile(heatmap, 100 - k*100)
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0-p1)

scores = []

for arr in ab:
    score = faithfulness(model, arr)
    scores.append(score)
df.at["Grad-CAM", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    """
    Measures the sparsity (number of important pixels).
    """
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable


scores = []

for arr in ab:
    heatmap = make_gradcam_heatmap(arr, model, last_conv_layer)
    score = explanation_complexity(heatmap)
    scores.append(score)
df.at["Grad-CAM", "Complexity"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """
    Compares similarity between original and randomized explanations.
    """
    from scipy.stats import spearmanr
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  


#RANDOM MODEL
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))
scores = []

for arr in ab:
    heatmap = make_gradcam_heatmap(arr, model, last_conv_layer)
    random_heatmap = make_gradcam_heatmap(arr, randomized_model, last_conv_layer)
    score = explanation_randomization_check(heatmap, random_heatmap)
    scores.append(score)

df.at["Grad-CAM", "Randomization"] = round(np.array(scores).mean(), 3)
display(df)

,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,-,-,-,-
Score-CAM,-,-,-,-
Integrated Gradients,-,-,-,-
SmoothGrad,-,-,-,-
Occlusion Sensitivity,-,-,-,-
GradCAM + Lung segment,-,-,-,-


<center><h1>Grad-CAM ++</h1></center>

In [19]:
#________________________________________________________________________________________________________________________GRAD-CAM++
#____________________________________________________________________________________________ROBUSTNESS


def explanation_robustness(original_explanation, image):
    epsilon=0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = make_gradcam_plus_plus_heatmap(perturbed_img, model, last_conv_layer)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)
scores = []
    
for arr in ab:
    heatmap = make_gradcam_heatmap(arr, model, last_conv_layer)
    score = explanation_robustness(heatmap, arr)
    scores.append(score)    
    
        
df.at["Grad-CAM++", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image):        
    k=0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    heatmap = make_gradcam_plus_plus_heatmap(image, model, last_conv_layer)
    threshold = np.percentile(heatmap, 100 - k*100)
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0-p1)

scores = []
for arr in ab:
    score = faithfulness(model, arr)
    scores.append(score)
df.at["Grad-CAM++", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    """
    Measures the sparsity (number of important pixels).
    """
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable


scores = []

for arr in ab:
    heatmap = make_gradcam_plus_plus_heatmap(arr, model, last_conv_layer)
    score = explanation_complexity(heatmap)
    scores.append(score)
df.at["Grad-CAM++", "Complexity"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """
    Compares similarity between original and randomized explanations.
    """
    from scipy.stats import spearmanr
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  # lower = more different, good for sanity check


#RANDOM MODEL
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))
scores = []

for arr in ab:
    heatmap = make_gradcam_plus_plus_heatmap(arr, model, last_conv_layer)
    random_heatmap = make_gradcam_heatmap(arr, randomized_model, last_conv_layer)
    score = explanation_randomization_check(heatmap, random_heatmap)
    scores.append(score)

df.at["Grad-CAM++", "Randomization"] = round(np.array(scores).mean(), 3)
display(df)

,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,-,-,-,-
Integrated Gradients,-,-,-,-
SmoothGrad,-,-,-,-
Occlusion Sensitivity,-,-,-,-
GradCAM + Lung segment,-,-,-,-


<center><h1>SCORE CAM </h1></center>

In [20]:
#________________________________________________________________________________________________________________________SCORE CAM
#____________________________________________________________________________________________ROBUSTNESS

new_list = images_arr

original_scorecams = [scorecam_heatmap(arr, model, last_conv_layer) for arr in new_list]

def explanation_robustness(original_explanation, image):
    epsilon = 0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = scorecam_heatmap(perturbed_img, model, last_conv_layer)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)

scores = []
for arr, heatmap in zip(new_list, original_scorecams):
    score = explanation_robustness(heatmap, arr)
    scores.append(score)
    
df.at["Score-CAM", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image, heatmap=None):        
    k = 0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    
    if heatmap is None:
        heatmap = scorecam_heatmap(image, model, last_conv_layer)
        
    threshold = np.percentile(heatmap, 100 - k*100)
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0 - p1)

scores = []
for arr, heatmap in zip(new_list, original_scorecams):
    score = faithfulness(model, arr, heatmap)
    scores.append(score)
    
df.at["Score-CAM", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    """Measures the sparsity (number of important pixels)."""
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable

scores = []
for heatmap in original_scorecams:
    score = explanation_complexity(heatmap)
    scores.append(score)
    
df.at["Score-CAM", "Complexity"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """Compares similarity between original and randomized explanations."""
    from scipy.stats import spearmanr
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  # lower = more different, good for sanity check

# Create randomized model (only once)
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))

# Precompute all random Grad-CAM heatmaps once
random_gradcams = [make_gradcam_heatmap(arr, randomized_model, last_conv_layer) for arr in new_list]

scores = []
for orig_heatmap, rand_heatmap in zip(original_scorecams, random_gradcams):
    score = explanation_randomization_check(orig_heatmap, rand_heatmap)
    scores.append(score)

df.at["Score-CAM", "Randomization"] = round(np.array(scores).mean(), 3)

display(df)

,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,1.628,0.127,0.994,0.069
Integrated Gradients,-,-,-,-
SmoothGrad,-,-,-,-
Occlusion Sensitivity,-,-,-,-
GradCAM + Lung segment,-,-,-,-


<center><h1>OCCLUSION</h1></center>

In [25]:
#________________________________________________________________________________________________________________________OCCLUSION
#____________________________________________________________________________________________ROBUSTNESS

listt = images_arr
original_heatmaps = [compute_occlusion_sensitivity(arr, model) for arr in listt]

def explanation_robustness(original_explanation, image):
    epsilon = 0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = compute_occlusion_sensitivity(perturbed_img, model)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)

scores = []
for arr, heatmap in zip(listt, original_heatmaps):
    score = explanation_robustness(heatmap, arr)
    scores.append(score)
    
df.at["Occlusion Sensitivity", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image, heatmap=None):        
    k = 0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    
    if heatmap is None:
        heatmap = compute_occlusion_sensitivity(image, model)
        
    threshold = np.percentile(heatmap, 100 - k*100)
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0 - p1)

scores = []
for arr, heatmap in zip(listt, original_heatmaps):
    score = faithfulness(model, arr, heatmap)
    scores.append(score)
    
df.at["Occlusion Sensitivity", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    """Measures the sparsity (number of important pixels)."""
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable

scores = []
for heatmap in original_heatmaps:
    score = explanation_complexity(heatmap)
    scores.append(score)
    
df.at["Occlusion Sensitivity", "Complexity"] = round(np.array(scores).mean(), 3)



#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """Compares similarity between original and randomized explanations."""
    from scipy.stats import spearmanr
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  # lower = more different, good for sanity check

# Create randomized model
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))

# Precompute random heatmaps once
random_heatmaps = [compute_occlusion_sensitivity(arr, randomized_model) for arr in listt]

scores = []
for orig_heatmap, rand_heatmap in zip(original_heatmaps, random_heatmaps):
    score = explanation_randomization_check(orig_heatmap, rand_heatmap)
    scores.append(score)

df.at["Occlusion Sensitivity", "Randomization"] = round(np.array(scores).mean(), 3)

display(df)

,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,1.628,0.127,0.994,0.069
Integrated Gradients,-,-,-,-
SmoothGrad,6.446,0.197,0.012,0.006
Occlusion Sensitivity,5.526,0.184,0.287,-0.022
GradCAM + Lung segment,-,-,-,-


<center><h1>SMOOTHGRAD</h1></center>

In [23]:
#________________________________________________________________________________________________________________________SMOOTHGRAD
#____________________________________________________________________________________________ROBUSTNESS
listt = []
listt = images_arr

# Precompute all heatmaps once
original_heatmaps = [make_smoothgrad_heatmap(arr, model) for arr in listt]

def explanation_robustness(original_explanation, image):
    epsilon = 0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = make_smoothgrad_heatmap(perturbed_img, model)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)

scores = []
for arr, heatmap in zip(listt, original_heatmaps):
    score = explanation_robustness(heatmap, arr)
    scores.append(score)
    
df.at["SmoothGrad", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image, heatmap=None):        
    k = 0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    
    if heatmap is None:
        heatmap = make_smoothgrad_heatmap(image, model)
        
    threshold = np.percentile(heatmap, 100 - k*100)
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0 - p1)

scores = []
for arr, heatmap in zip(listt, original_heatmaps):
    score = faithfulness(model, arr, heatmap)
    scores.append(score)
    
df.at["SmoothGrad", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    """Measures the sparsity (number of important pixels)."""
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable

scores = []
for heatmap in original_heatmaps:
    score = explanation_complexity(heatmap)
    scores.append(score)
    
df.at["SmoothGrad", "Complexity"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """Compares similarity between original and randomized explanations."""
    from scipy.stats import spearmanr
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  # lower = more different, good for sanity check

# Create randomized model
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))

# Precompute random heatmaps once
random_heatmaps = [make_smoothgrad_heatmap(arr, randomized_model) for arr in listt]

scores = []
for orig_heatmap, rand_heatmap in zip(original_heatmaps, random_heatmaps):
    score = explanation_randomization_check(orig_heatmap, rand_heatmap)
    scores.append(score)
# Calculate mean of non-nan values
valid_scores = [x for x in scores if not np.isnan(x)]
mean_score = np.mean(valid_scores)

scores = [mean_score if np.isnan(x) else x for x in scores]
df.at["SmoothGrad", "Randomization"] = round(np.array(scores).mean(), 3)



/tmp/ipykernel_32/2244567500.py:70: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = spearmanr(orig_flat, rand_flat)


,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,1.628,0.127,0.994,0.069
Integrated Gradients,-,-,-,-
SmoothGrad,6.446,0.197,0.012,0.006
Occlusion Sensitivity,5.526,0.184,0.287,-
GradCAM + Lung segment,-,-,-,-


In [26]:
display(df)

,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,1.628,0.127,0.994,0.069
Integrated Gradients,-,-,-,-
SmoothGrad,6.446,0.197,0.012,0.006
Occlusion Sensitivity,5.526,0.184,0.287,-0.022
GradCAM + Lung segment,-,-,-,-


<center><h1>INTEGRATED GRADIENTS</h1></center>

In [27]:
#________________________________________________________________________________________________________________________INTEGRATED GRADIENTS
#____________________________________________________________________________________________ROBUSTNESS
listt = []
listt = images_arr

# Precompute all heatmaps once
original_heatmaps = [make_integrated_gradients_heatmap(arr, model) for arr in listt]
def explanation_robustness(original_explanation, image):
    epsilon = 0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = make_integrated_gradients_heatmap(perturbed_img, model)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)

scores = []
for arr, heatmap in zip(listt, original_heatmaps):
    score = explanation_robustness(heatmap, arr)
    scores.append(score)
    
df.at["Integrated Gradients", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image, heatmap=None):        
    k = 0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    
    if heatmap is None:
        heatmap = make_integrated_gradients_heatmap(image, model)
        
    threshold = np.percentile(heatmap, 100 - k*100)
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0 - p1)

scores = []
for arr, heatmap in zip(listt, original_heatmaps):
    score = faithfulness(model, arr, heatmap)
    scores.append(score)
    
df.at["Integrated Gradients", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    """Measures the sparsity (number of important pixels)."""
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable

scores = []
for heatmap in original_heatmaps:
    score = explanation_complexity(heatmap)
    scores.append(score)
    
df.at["Integrated Gradients", "Complexity"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """Compares similarity between original and randomized explanations."""
    from scipy.stats import spearmanr
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  # lower = more different, good for sanity check

# Create randomized model
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))

# Precompute random heatmaps once
random_heatmaps = [make_integrated_gradients_heatmap(arr, randomized_model) for arr in listt]

scores = []
for orig_heatmap, rand_heatmap in zip(original_heatmaps, random_heatmaps):
    score = explanation_randomization_check(orig_heatmap, rand_heatmap)
    scores.append(score)

df.at["Integrated Gradients", "Randomization"] = round(np.array(scores).mean(), 3)

display(df)

hello


,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,1.628,0.127,0.994,0.069
Integrated Gradients,21.68,0.25,1.0,0.0
SmoothGrad,6.446,0.197,0.012,0.006
Occlusion Sensitivity,5.526,0.184,0.287,-0.022
GradCAM + Lung segment,-,-,-,-


<center><h1>GRAD-CAM  +  LUNG SEGMENTATION</h1></center>

In [7]:

def mask_lung(img_path, s_model=segmentation_model):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    #Segmentation model
    original = cv2.resize(img, (256, 256))
    original = original.astype("float32") / 255.0
    original = np.expand_dims(original, axis=-1) 
    original = np.expand_dims(original, axis=0)
    git_seg = s_model.predict(original, verbose=0)
    git_seg = np.squeeze(git_seg)

    #openCV
    equalized = cv2.equalizeHist(img)

    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
    first = cv2.resize(blurred, (256, 256))
    first = first.astype("float32") / 255.0
    first = np.expand_dims(first, axis=-1) 
    first = np.expand_dims(first, axis=0)
    first_seg = s_model.predict(first, verbose=0)
    first_seg = np.squeeze(first_seg) 

    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)

    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    second = cv2.resize(opened, (256, 256))
    second = second.astype("float32") / 255.0
    second = np.expand_dims(second, axis=-1) 
    second = np.expand_dims(second, axis=0)
    second_seg = s_model.predict(second, verbose=0)
    second_seg = np.squeeze(second_seg)

    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lung_mask = np.zeros_like(img)
    cv2.drawContours(lung_mask, contours, -1, 255, thickness=cv2.FILLED)
    
    segmented = cv2.bitwise_and(img, img, mask=lung_mask)
    third = cv2.resize(segmented, (256, 256))
    third = third.astype("float32") / 255.0
    third = np.expand_dims(third, axis=-1) 
    third = np.expand_dims(third, axis=0)
    third_seg = s_model.predict(third, verbose=0)
    third_seg = np.squeeze(third_seg)

    mask = np.maximum.reduce([git_seg, first_seg, second_seg, third_seg])
    mask = np.expand_dims(mask, axis=-1) 
    mask = tf.image.resize(mask, [512, 512], method='bilinear')
    return mask
    
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = tf.image.resize(
        heatmap[..., tf.newaxis],  
        (512, 512),
        method=tf.image.ResizeMethod.BILINEAR  
    )
    return heatmap.numpy()


def meaningful_expl_heatmap(img_array, img_path, model):
    mask = mask_lung(img_path)
    img_array = get_arr_img(img_path)
    grad_cam = make_gradcam_heatmap(img_array, model, last_conv_layer)
    masked_heatmap = grad_cam * mask
    return masked_heatmap.numpy()

In [ ]:
#________________________________________________________________________________________________________________________GRAD-CAM+ LUNG SEGMENTATION
#____________________________________________________________________________________________ROBUSTNESS

ab = []
with open("/kaggle/input/random-paths/random_paths.txt", "r") as file:
    for line in file:
        path = line.strip()
        ab.append(path)

def explanation_robustness(original_explanation, image, path, t_model):
    epsilon=0.01
    noise = np.random.normal(loc=0.0, scale=epsilon, size=image.shape)
    perturbed_img = np.clip(image + noise, 0, 1)
    expl = meaningful_expl_heatmap(perturbed_img, path, t_model)
    diffs = np.linalg.norm(original_explanation - expl)
    return round(np.mean(diffs), 3)
scores = []
    
for p in ab:
    arr = get_arr_img(p)
    heatmap = meaningful_expl_heatmap(arr, p, model)
    score = explanation_robustness(heatmap, arr, p, model)
    scores.append(score)    
    
        
df.at["GradCAM + Lung segment", "Robustness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________FAITHFULNESS
def faithfulness(model, image, path, t_model):        
    k=0.20
    p0 = round(model.predict(image, verbose=0)[0][0], 3)
    heatmap = meaningful_expl_heatmap(image, path, t_model)
    threshold = np.percentile(heatmap, 100 - k*100)
    #heatmap = heatmap.numpy()
    mask = (heatmap >= threshold).astype(bool)
        
    image_deleted = np.copy(tf.squeeze(image).numpy())
    image_deleted = np.expand_dims(image_deleted, axis=-1) 
    image_deleted[mask] = 0 
    
    image_deleted = np.expand_dims(image_deleted, 0)
    p1 = round(model.predict(image_deleted, verbose=0)[0][0], 3)
    return abs(p0-p1)

scores = []

for p in ab:
    arr = get_arr_img(p)
    score = faithfulness(model, arr, p, model)
    scores.append(score)
df.at["GradCAM + Lung segment", "Faithfulness"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________COMPLEXITY
def explanation_complexity(explanation, threshold=0.1):
    #explanation = explanation.numpy() 
    total_pixels = explanation.size
    important_pixels = np.sum(explanation > threshold)
    return important_pixels / total_pixels  # lower = more interpretable


scores = []

for p in ab:
    arr = get_arr_img(p)
    heatmap = meaningful_expl_heatmap(arr, p, model)
    score = explanation_complexity(heatmap)
    scores.append(score)
df.at["GradCAM + Lung segment", "Complexity"] = round(np.array(scores).mean(), 3)

#____________________________________________________________________________________________RANDOMIZATION
def explanation_randomization_check(original_expl, randomized_expl):
    """
    Compares similarity between original and randomized explanations.
    """
    from scipy.stats import spearmanr
    if hasattr(original_expl, 'numpy'):
        original_expl = original_expl.numpy()
        randomized_expl = randomized_expl.numpy()
    orig_flat = original_expl.flatten()
    rand_flat = randomized_expl.flatten()
    corr, _ = spearmanr(orig_flat, rand_flat)
    return corr  # lower = more different, good for sanity check


#RANDOM MODEL
randomized_model = tf.keras.models.clone_model(model)
randomized_model.build((None, 512, 512, 1))
for layer in randomized_model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
        layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))
scores = []

for p in ab:
    arr = get_arr_img(p)
    heatmap = meaningful_expl_heatmap(arr, p, model)
    random_heatmap = meaningful_expl_heatmap(arr, p, randomized_model)
    score = explanation_randomization_check(heatmap, random_heatmap)
    scores.append(score)

df.at["GradCAM + Lung segment", "Randomization"] = round(np.array(scores).mean(), 3)

In [12]:
display(df)


,Robustness,Faithfulness,Complexity,Randomization
Grad-CAM,7.878000,0.577,0.522,-0.144
Grad-CAM++,135.330994,0.302,0.534,0.016
Score-CAM,1.628000,0.127,0.994,0.069
Integrated Gradients,21.680000,0.250,1.000,0.000
SmoothGrad,6.446000,0.197,0.012,0.006
Occlusion Sensitivity,5.526000,0.184,0.287,-0.022
GradCAM + Lung segment,0.330000,0.737,0.049,-0.208
